## Imports

In [4]:
pip install ftfy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 480.1 kB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 670.6 kB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [16]:
import numpy as np
import pandas as pd
import re
from ftfy import fix_text
import unidecode
import json

In [17]:
df = pd.read_json("gg2013.json")

# Extract user information into separate columns
df['user_screen_name'] = df['user'].apply(lambda x: x['screen_name'])
df['user_id'] = df['user'].apply(lambda x: x['id'])

# Drop the original 'user' column as we've extracted the needed information
df = df.drop('user', axis=1)

# Convert timestamp_ms to datetime
df['timestamp'] = pd.to_datetime(df['timestamp_ms'], unit='ms')

# Drop the original timestamp_ms column
df = df.drop('timestamp_ms', axis=1)

# Reorder columns for better readability
df = df[['id', 'timestamp', 'user_id', 'user_screen_name', 'text']]

# Display the first few rows of the DataFrame
print(df.head())

                   id           timestamp    user_id user_screen_name  \
0  290620657987887104 2013-01-14 00:45:38  557374298        Dozaaa_xo   
1  290620657887219713 2013-01-14 00:45:38   14648726     theAmberShow   
2  290620657828524032 2013-01-14 00:45:38   35498686         SweetyPW   
3  290620657799159809 2013-01-14 00:45:38  144430208   _NicoleEdwards   
4  290620657778188288 2013-01-14 00:45:38  134953223    lolaogunnaike   

                                                text  
0             JLo's dress! #eredcarpet #GoldenGlobes  
1  What's making Sofia Vergara's boobs stay like ...  
2  RT @FabSugar: Kerry Washington is EVERYTHING. ...  
3     Anne Hathaway has got me living. #GoldenGlobes  
4  Jennifer Lopez's lace dress? Thoughts? #Golden...  


In [18]:
def preprocess_text(text):
    # Fix encoding issues (ampersands, etc.) using ftfy
    text = fix_text(text)
    
    # Remove non-ASCII characters (emojis, unicode symbols) using unidecode
    text = unidecode.unidecode(text)
    
    # Extract hashtags and links
    hashtags = re.findall(r'#\w+', text)  # Extract hashtags
    links = re.findall(r'http[s]?://\S+', text)  # Extract URLs
    
    # Remove hashtags and links from the original text
    text = re.sub(r'#\w+', '', text)  # Remove hashtags
    text = re.sub(r'http[s]?://\S+', '', text)  # Remove URLs
    
    # Remove extra whitespace, tabs, and newlines (substitute with single spaces)
    # If we want to keep tabs/newline characters: text = re.sub(' +', ' ', text)
    text = " ".join(text.split())
    
    return text, hashtags, links

# Apply preprocessing
df[['cleaned_text', 'hashtags', 'links']] = df['text'].apply(
    lambda x: pd.Series(preprocess_text(x))
)

# Display the updated DataFrame
df[['text', 'cleaned_text', 'hashtags', 'links']].head()

,text,cleaned_text,hashtags,links
0,JLo's dress! #eredcarpet #GoldenGlobes,JLo's dress!,"[#eredcarpet, #GoldenGlobes]",[]
1,What's making Sofia Vergara's boobs stay like ...,What's making Sofia Vergara's boobs stay like ...,[#GoldenGlobes],[]
2,RT @FabSugar: Kerry Washington is EVERYTHING. ...,RT @FabSugar: Kerry Washington is EVERYTHING. ...,[#goldenglobes],[http://t.co/3feH7MvO]
3,Anne Hathaway has got me living. #GoldenGlobes,Anne Hathaway has got me living.,[#GoldenGlobes],[]
4,Jennifer Lopez's lace dress? Thoughts? #Golden...,Jennifer Lopez's lace dress? Thoughts?,[#GoldenGlobes],[]


In [19]:
df = df.sort_values(by='timestamp')
df.head()

,id,timestamp,user_id,user_screen_name,text,cleaned_text,hashtags,links
0,290620657987887104,2013-01-14 00:45:38,557374298,Dozaaa_xo,JLo's dress! #eredcarpet #GoldenGlobes,JLo's dress!,"[#eredcarpet, #GoldenGlobes]",[]
25,290620656456982528,2013-01-14 00:45:38,175201062,TVDPLL,My Baby Leo DiCaprio.aaaaaah estava esperando ...,My Baby Leo DiCaprio.aaaaaah estava esperando ...,[#GoldenGlobes],[]
26,290620656394051584,2013-01-14 00:45:38,324397071,FringePhalanges,Kerry Washington is perfect!!!! #GoldenGlobes,Kerry Washington is perfect!!!!,[#GoldenGlobes],[]
27,290620656314351616,2013-01-14 00:45:38,71066469,ACaryGrantFan,It should be illegal to look as good as Ewan M...,It should be illegal to look as good as Ewan M...,[#GoldenGlobes],[]
28,290620656276631552,2013-01-14 00:45:38,818609779,OnScreenStyle,.@kerrywashington's nude &amp; sheer = yes. Je...,.@kerrywashington's nude & sheer = yes. Jennif...,[#goldenglobes],[]


In [20]:
df.drop(columns='text', inplace=True)

In [21]:
df.head()

,id,timestamp,user_id,user_screen_name,cleaned_text,hashtags,links
0,290620657987887104,2013-01-14 00:45:38,557374298,Dozaaa_xo,JLo's dress!,"[#eredcarpet, #GoldenGlobes]",[]
25,290620656456982528,2013-01-14 00:45:38,175201062,TVDPLL,My Baby Leo DiCaprio.aaaaaah estava esperando ...,[#GoldenGlobes],[]
26,290620656394051584,2013-01-14 00:45:38,324397071,FringePhalanges,Kerry Washington is perfect!!!!,[#GoldenGlobes],[]
27,290620656314351616,2013-01-14 00:45:38,71066469,ACaryGrantFan,It should be illegal to look as good as Ewan M...,[#GoldenGlobes],[]
28,290620656276631552,2013-01-14 00:45:38,818609779,OnScreenStyle,.@kerrywashington's nude & sheer = yes. Jennif...,[#goldenglobes],[]


In [29]:
df[df.cleaned_text.str.contains('wins the award for')]

,id,timestamp,user_id,user_screen_name,cleaned_text,hashtags,links
14005,290626769285951488,2013-01-14 01:09:56,18695901,NikkiBurdine,RT @nbc: Christoph Waltz wins the award for Be...,[#GoldenGlobes],[]
14030,290626797404581890,2013-01-14 01:10:02,48137468,nicbleidorn,RT @nbc: Christoph Waltz wins the award for Be...,[#GoldenGlobes],[]
14027,290626797480062976,2013-01-14 01:10:02,14189835,nbcdfwscene,RT @nbc: Christoph Waltz wins the award for Be...,[#GoldenGlobes],[]
14846,290626881047379968,2013-01-14 01:10:22,25917137,Eli_bebe_DBZ,RT @nbc: Christoph Waltz wins the award for Be...,[#GoldenGlobes],[]
14878,290626886130864128,2013-01-14 01:10:23,26292984,ProducerClaire,RT @nbc: Christoph Waltz wins the award for Be...,[#GoldenGlobes],[]
15132,290626910810152960,2013-01-14 01:10:29,33652676,a_loulou,RT @nbc: Christoph Waltz wins the award for Be...,[#GoldenGlobes],[]
15345,290626935166476288,2013-01-14 01:10:35,281255206,ELBAROBERTA,RT @nbc: Christoph Waltz wins the award for Be...,[#GoldenGlobes],[]
16257,290627126762287104,2013-01-14 01:11:21,787807824,coloredwthchaos,RT @nbc: Christoph Waltz wins the award for Be...,[#GoldenGlobes],[]
16642,290627257100271616,2013-01-14 01:11:52,53750818,michaelhirsh,Christoph Waltz wins the award for The Horribl...,[#goldenglobes],[]
23151,290629132860153858,2013-01-14 01:19:19,109618925,britangel21,RT @nbc: Christoph Waltz wins the award for Be...,[#GoldenGlobes],[]


## Output

In [4]:
# Sample inputs
host = "Seth Meyers"
awards = {
    "Best Motion Picture - Drama": {
        "Presenters": ["Barbara Streisand"],
        "Nominees": [
            "Three Billboards Outside Ebbing, Missouri", 
            "Call Me by Your Name", 
            "Dunkirk", 
            "The Post", 
            "The Shape of Water"
        ],
        "Winner": "Three Billboards Outside Ebbing, Missouri"
    },
    "Best Motion Picture - Musical or Comedy": {
        "Presenters": ["Alicia Vikander", "Michael Keaton"],
        "Nominees": [
            "Lady Bird", 
            "The Disaster Artist", 
            "Get Out", 
            "The Greatest Showman", 
            "I, Tonya"
        ],
        "Winner": "Lady Bird"
    }
}

def format_human_readable(host, awards):
    result = f"Host: {host}\n\n"
    for award_name, details in awards.items():
        result += f"Award: {award_name}\n"
        result += f"Presenters: {', '.join(details['Presenters'])}\n"
        result += f'''Nominees: {', '.join([f'"{nominee}"' for nominee in details['Nominees']])}\n'''
        result += f"Winner: \"{details['Winner']}\"\n\n"
    return result

def format_json(host, awards):
    json_result = {
        "Host": host
    }
    json_result.update(awards)
    return json.dumps(json_result, indent=4)


human_readable_output = format_human_readable(host, awards)
json_output = format_json(host, awards)

print("Human-Readable Format:\n")
print(human_readable_output)

print("JSON Format:\n")
print(json_output)

Human-Readable Format:

Host: Seth Meyers

Award: Best Motion Picture - Drama
Presenters: Barbara Streisand
Nominees: "Three Billboards Outside Ebbing, Missouri", "Call Me by Your Name", "Dunkirk", "The Post", "The Shape of Water"
Winner: "Three Billboards Outside Ebbing, Missouri"

Award: Best Motion Picture - Musical or Comedy
Presenters: Alicia Vikander, Michael Keaton
Nominees: "Lady Bird", "The Disaster Artist", "Get Out", "The Greatest Showman", "I, Tonya"
Winner: "Lady Bird"


JSON Format:

{
    "Host": "Seth Meyers",
    "Best Motion Picture - Drama": {
        "Presenters": [
            "Barbara Streisand"
        ],
        "Nominees": [
            "Three Billboards Outside Ebbing, Missouri",
            "Call Me by Your Name",
            "Dunkirk",
            "The Post",
            "The Shape of Water"
        ],
        "Winner": "Three Billboards Outside Ebbing, Missouri"
    },
    "Best Motion Picture - Musical or Comedy": {
        "Presenters": [
            "Ali